In [1]:
!pip install catboost
!pip install scikit-learn --upgrade

     |████████████████████████████████| 67.4 MB 51 kB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
# If you have installation questions, please reach out

import pandas as pd # data storage
import catboost as cats # graident boosting 

from catboost import CatBoostRegressor, Pool

import datetime
import numpy as np  # math and stuff
import matplotlib.pyplot as plt # plotting utility
import sklearn # ML and stats

print('catboost ver:', cats.__version__)
print('scikit ver:', sklearn.__version__)

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import max_error, mean_squared_error, median_absolute_error

catboost ver: 0.26.1
scikit ver: 0.24.2


In [3]:

df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS1_Merged_dataset_imputed_08_23_2021.csv')

#df3 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS2_offset_imputed_test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

print(df.columns.values) # printing all column names

df.describe()

['depth_ft' 'USGS_ID' 'CAL' 'GR' 'DT' 'SP' 'DENS' 'PE' 'RESD' 'PHIN'
 'PHID' 'PE_smooth' 'GR_smooth' 'gz_linear_interp' 'gz_pchip_interp'
 'perm_air_md' 'perm_klink_md' 'He_por' 'horz_perm_md' 'effective_por'
 'por_percent' 'stress_swanson_perm_md' 'sample' 'Na' 'Na Err' 'Mg'
 'Mg Err' 'Al' 'Al Err' 'Si' 'Si Err' 'P' 'P Err' 'S' 'S Err' 'K' 'K Err'
 'Ca' 'Ca Err' 'Ti' 'Ti Err' 'Cr' 'Cr Err' 'Mn' 'Mn Err' 'Fe' 'Fe Err'
 'Co' 'Co Err' 'Ni' 'Ni Err' 'Cu' 'Cu Err' 'Zn' 'Zn Err' 'Ba' 'Ba Err']


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,508.000000,510.000000,510.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5397.863358,0.466364,0.329083,0.429834,0.811770,0.614101,0.350337,1.016528,0.170502,0.170502,0.350431,0.329237,4.913576,4.910244,81.002187,73.797216,14.267039,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5106.156677,0.194187,0.237374,0.264391,0.158002,0.214754,0.188003,1.493488,0.223270,0.223270,0.180167,0.229605,1.738128,1.751499,173.917355,161.251984,9.365678,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003752,0.029593,0.000000,0.000000,-0.744314,0.000570,0.000570,0.004477,0.000258,1.204296,1.199972,0.001000,0.000000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.725000,0.357143,0.124088,0.169980,0.746766,0.422457,0.214025,-0.049160,0.004607,0.004607,0.216940,0.133040,3.538724,3.531571,0.040750,0.020250,8.300000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1614.650000,0.554827,0.279201,0.467738,0.865085,0.605957,0.322729,0.807690,0.005607,0.005607,0.329108,0.287389,4.307541,4.256191,0.065500,0.037500,9.300000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9649.575000,0.602566,0.530078,0.663285,0.922367,0.830974,0.468130,1.394680,0.330625,0.330625,0.461883,0.522949,6.615175,6.633493,71.375000,59.675000,26.610000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.0

# Split wells

In [5]:
wellz = df.USGS_ID.unique()
wellz

array(['F028', 'E952', 'E945', 'E974', 'F041', 'F042', 'E934', 'E997',
       'S821', 'CEPO', 'S179', 'PDRMT'], dtype=object)

In [6]:
for x in wellz[:2]:
  x

In [7]:
x

'E952'

In [8]:
for x in wellz[:2]:
  #solo well df
  df_well = df[df['USGS_ID']== x]
  #dfz = df[df['USGS_ID']!= x]
  print(x)
  df_well

F028
E952


In [9]:
df_well

,depth_ft,USGS_ID,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,sample,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
1161,10215.5,E952,0.553086,0.583566,0.112154,0.639183,0.690498,0.274851,-0.312524,0.259259,0.259259,0.292068,0.569672,6.9549,6.9386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1162,10215.6,E952,0.553102,0.578972,0.112085,0.639200,0.690407,0.275984,-0.309212,0.261235,0.261235,0.290673,0.565131,6.9536,6.9439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1163,10215.7,E952,0.553119,0.574377,0.112015,0.639217,0.690317,0.277116,-0.305900,0.263210,0.263210,0.289191,0.560551,6.9524,6.9474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1164,10215.8,E952,0.553135,0.569782,0.111946,0.639234,0.690226,0.278248,-0.302589,0.265185,0.265185,0.287624,0.555931,6.9511,6.9493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1165,10215.9,E952,0.553152,0.565187,0.111877,0.639251,0.690136,0.279380,-0.299277,0.267160,0.267160,0.286080,0.551432,6.9499,6.9499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,10272.0,E952,0.555325,0.590256,0.143077,0.656968,0.679186,0.258184,0.537147,0.320988,0.320988,0.249421,0.594302,6.7803,6.7803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727,10272.1,E952,0.555524,0.591139,0.142854,0.657679,0.678552,0.258939,0.506188,0.321975,0.321975,0.249243,0.594869,6.7812,6.7812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1728,10272.2,E952,0.555723,0.592023,0.142631,0.658390,0.677919,0.259694,0.475229,0.322963,0.322963,0.249177,0.595371,6.7821,6.7821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1729,10272.3,E952,0.555922,0.592906,0.142407,0.659100,0.677285,0.260448,0.444270,0.323951,0.323951,0.249222,0.595809,6.7830,6.7830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for z in wellz:
  #solo well df
  df_well = df[df['USGS_ID']==z]
  dfz = df[df['USGS_ID']!=z]

  dataset = dfz[[
    'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
    'RESD', 'PHIN', 'PHID', 'GR_smooth', 
    'PE_smooth',
    'gz_pchip_interp' # Trying to predict gz_pchip_interp
    ]]

  dataset.replace('NaN',np.nan, regex=True, inplace=True)# Should be good already

  # Features we will use for prediction
  X = dataset[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

  # What we are trying to predict
  Y = dataset[['gz_pchip_interp']]
  Y_array = np.array(Y.values)

  seed = 42 # random seed is only used if you want to compare exact answers with friends 
  test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

  #test train split
  X_train, X_test, y_train, y_test = train_test_split(X.values, Y_array, test_size=test_size)

  export_name = '_single_well.csv'
  max_iter=700

  # def catboost_gz(X_train, X_test, y_train, y_test, export_name, max_iter=500):

  model = CatBoostRegressor(objective='RMSE',
                            task_type='GPU',
                            iterations=max_iter)
  
  model.fit(X_train, y_train, verbose=1000 )
  preds = model.predict(X_test)

  # Stats
  rmse = mean_squared_error(y_test, preds, squared=False)
  print("Root Mean Squared Error: %f" % (rmse))
  max = max_error(y_test, preds)
  print("Max Error: %f" % (max))
  MAE= median_absolute_error(y_test, preds)
  print("Median Abs Error: %f" % (max))

  # Let's Tune
  grid = {'learning_rate': [0.1, 0.2, 0.4],
        'depth': [6, 8, 10, 12],
        'l2_leaf_reg': [2, 3, 4]}

  model_grid = CatBoostRegressor(objective='RMSE',
                                 task_type='GPU',
                                 iterations=max_iter)

  # Grid Search
  grid_search_result = model_grid.grid_search(grid, 
                                            X=X_train, 
                                            y=y_train, 
                                            cv=5,
                                            verbose=False)
  model2 = CatBoostRegressor(objective='RMSE',
                           depth=grid_search_result['params']['depth'],
                           l2_leaf_reg=grid_search_result['params']['l2_leaf_reg'],
                           learning_rate=grid_search_result['params']['learning_rate'],
                           iterations=max_iter)

  model2.fit(X_train, y_train, verbose=500)
  preds2 = model2.predict(X_test)

  # Stats for the second round
  rmse2 = mean_squared_error(y_test, preds2, squared=False)
  print("Root Mean Squared Error: %f" % (rmse2))
  max2 = max_error(y_test, preds2)
  print("Max Error: %f" % (max2))
  MAE2= median_absolute_error(y_test, preds2)
  print("Median Abs Error: %f" % (MAE2))

  # Solo Well fun
  dataset2 = df_well[[
    'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
    'RESD', 'PHIN', 'PHID', 'GR_smooth', 
    'PE_smooth',
    'gz_pchip_interp' # Trying to predict gz_pchip_interp
    ]]

  dataset2.replace('NaN',np.nan, regex=True, inplace=True)# Should be good already
 
  # Features we will use for prediction
  X2 = dataset2[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

  # What we are trying to predict
  Y2 = dataset2[['gz_pchip_interp']]
  Y2_array = np.array(Y2.values)
  
  # Make predictions from previous model
  preds3 = model2.predict(X2)

  # Stats for the second round
  rmse3 = mean_squared_error(Y2_array, preds3, squared=False)
  print("Root Mean Squared Error: %f" % (rmse3))
  max3 = max_error(Y2_array, preds3)
  print("Max Error: %f" % (max3))
  MAE3= median_absolute_error(Y2_array, preds3)
  print("Median Abs Error: %f" % (MAE3))

  x = datetime.datetime.now()

  d = {'well': [ z, z, z],
      'target': [Y.columns.values, Y.columns.values, Y.columns.values],
     'RMSE': [rmse, rmse2, rmse3],
     'MAE': [MAE, MAE2, MAE3],
     'MaxError': [max, max2, max3], 
     'iter': [max_iter, max_iter, max_iter],
     'day': [x.day, x.day, x.day], 
     'month':[x.month, x.month, x.month], 
     'year':[x.year, x.year, x.year],
     'model':['catboost', 'catboost', 'catboost'],
     'version':[cats.__version__, cats.__version__ , cats.__version__ ]}

  results = pd.DataFrame(data=d)

  filepath = 'drive/My Drive/1_lewis_research/analysis/experiments/catboost/catboost_results/'
  results.to_csv(filepath+ z + export_name)

 # return results

Output hidden; open in https://colab.research.google.com to view.

In [11]:
df

,depth_ft,USGS_ID,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,sample,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
0,7741.0,F028,0.598388,0.139541,0.062974,0.578901,0.834201,0.355534,1.817681,0.001734,0.001734,0.354183,0.161796,4.0313,4.0320,NaN,NaN,NaN,NaN,NaN,14.5,5.55,1500675-Mudrock Dual,1.2770,0.0263,0.6987,0.0116,2.8101,0.0112,33.9921,0.0332,0.0826,0.0021,0.1116,0.0011,0.9947,0.0045,4.5640,0.0073,0.1641,0.0013,0.0085,0.0001,0.0186,0.0008,1.4531,0.0029,0.0011,0.0,0.0044,0.0001,0.0,0.0008,0.0023,0.0008,0.0663,0.0049
1,7741.1,F028,0.599032,0.140214,0.064365,0.578108,0.833761,0.354277,1.792578,0.001752,0.001752,0.352506,0.160427,3.9834,3.9834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7741.2,F028,0.599694,0.141007,0.065247,0.577456,0.833524,0.352951,1.770195,0.001744,0.001744,0.351397,0.159515,3.9837,3.9836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7741.3,F028,0.600342,0.141384,0.064707,0.576524,0.833115,0.351924,1.745634,0.001737,0.001737,0.350745,0.159056,3.9840,3.9840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7741.4,F028,0.600995,0.142250,0.064707,0.576234,0.832757,0.350847,1.723410,0.001729,0.001729,0.350246,0.159017,3.9843,3.9843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12025,13378.0,PDRMT,0.080605,0.755416,0.320083,0.502753,0.617269,0.421889,-0.186078,0.395805,0.395805,0.462766,0.764521,4.3717,4.3658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500503-Mudrock Dual,0.5632,0.0196,0.5084,0.0102,1.8027,0.0094,35.4358,0.0311,0.1020,0.0024,0.2357,0.0015,0.5846,0.0034,4.3886,0.0066,0.1207,0.0012,0.0114,0.0002,0.0153,0.0007,1.0049,0.0024,0.0008,0.0,0.0049,0.0001,0.0,0.0007,0.0034,0.0008,0.1478,0.0056
12026,13378.1,PDRMT,0.080677,0.759066,0.319364,0.502550,0.622949,0.425651,-0.175045,0.386929,0.386929,0.467023,0.765645,6.6922,6.7092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12027,13378.2,PDRMT,0.080749,0.763390,0.318322,0.502314,0.628529,0.428994,-0.163882,0.378053,0.378053,0.471810,0.766522,6.7190,6.7313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12028,13378.3,PDRMT,0.080822,0.767160,0.319374,0.501919,0.636449,0.432019,-0.152552,0.369177,0.369177,0.477121,0.767302,4.3369,4.3273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Now Run it on F028

In [12]:
dataset2 = df_F028[[
    'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
    'RESD', 'PHIN', 'PHID', 'GR_smooth', 
    'PE_smooth',
    'gz_pchip_interp' # Trying to predict gz_pchip_interp
]]

dataset2.replace('NaN',np.nan, regex=True, inplace=True)# Should be good already
# dataset = dataset.dropna() # not needed

dataset2.head(3)

NameError: ignored

In [ ]:
# Features we will use for prediction
X2 = dataset2[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y2 = dataset2[['gz_pchip_interp']]

Y2_array = np.array(Y2.values)

In [ ]:
preds3 = model2.predict(X2)
preds3

In [ ]:
# Stats for the second round
rmse2 = mean_squared_error(Y2_array, preds3, squared=False)
print("Root Mean Squared Error: %f" % (rmse2))
max2 = max_error(Y2_array, preds3)
print("Max Error: %f" % (max2))
MAE2= median_absolute_error(Y2_array, preds3)
print("Median Abs Error: %f" % (MAE2))

In [ ]:
Y2.gz_pchip_interp

In [ ]:
np.shape(Y2.gz_pchip_interp.values)

In [ ]:
np.shape(preds3[:])

In [ ]:
smooth_window = np.ones(7)/7

In [ ]:
np.convolve(smooth_window, preds3, mode='same')

In [ ]:
window = 3
smooth_window = np.ones(window)/window

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(Y2.gz_pchip_interp.values)
#plt.plot(preds3)
plt.plot(np.convolve(smooth_window, preds3, mode='same'))
#plt.plot(np.convolve(smooth_window, Y2.gz_pchip_interp.values, mode='same'))
plt.grid(True)


In [ ]:
yo = (preds3 - preds.min())/preds3.max()
mo = ((Y2 - Y2.min())/Y2.max())

In [ ]:
plt.plot(1-yo, color='black')
plt.plot(mo, color='g')

In [ ]:
yo-mo.T

In [ ]:
ted = (preds3 - Y2_array.T)